In [270]:
from trackml.dataset import load_event
from trackml.dataset import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import random

def proc_arr(x):
    for i in range(len(x) - 1):
        block_stat.ix[x[i], x[i+1]] += 1
        
def proc_arr_next_hit(x):
    for i in range(len(x) - 1):
        next_hits.ix[x[i], 'next_hit'] = x[i+1]

In [261]:
cnt = 0
X_train = pd.DataFrame()
X_val = pd.DataFrame()

for event_id, hits, cells, particles, truth in load_dataset('data/train_5.zip'):
    if cnt == 10:
        break
    
    
    
    truth_hits = pd.concat([hits, truth.drop(['hit_id'], axis=1)], axis=1)
    truth_hits['dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2 + x[3]**2, axis=1)
    truth_hits['rad_dist'] = truth_hits.apply(lambda x: x[1]**2 + x[2]**2, axis=1)
    
    truth_hits = truth_hits[truth_hits.particle_id != 0].sort_values(['particle_id', 'rad_dist'])
    truth_hits['block'] = truth_hits['volume_id'].astype(str) + "_" + truth_hits['layer_id'].astype(str)
    
    

    cell_stats = cells.groupby('hit_id')['value'].mean().reset_index().rename({'value':'value_mean'},axis=1)

    tmp = cells.groupby('hit_id')['value'].max().reset_index().rename({'value':'value_max'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].min().reset_index().rename({'value':'value_min'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].sum().reset_index().rename({'value':'value_sum'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].std().reset_index().rename({'value':'value_std'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')

    tmp = cells.groupby('hit_id')['value'].count().reset_index().rename({'value':'value_count'},axis=1)
    cell_stats = pd.merge(cell_stats, tmp, on='hit_id', how='left')    
    
    cell_cols = list(cell_stats.columns.values[1:])  
    
    
    truth_hits = pd.merge(truth_hits, cell_stats, on='hit_id', how='left')
    
    
    next_hits = pd.DataFrame(truth_hits['hit_id'].unique()).rename({0:'hit_id'},axis=1)
    next_hits['next_hit'] = np.nan

    truth_hits['next_particle'] = list(truth_hits['particle_id'].values[1:]) + [-1]
    truth_hits['next_hit'] = list(truth_hits['hit_id'].values[1:]) + [-1]
    truth_hits['not_end_hit'] = truth_hits['particle_id'] == truth_hits['next_particle']
    truth_hits['next_hit']  = truth_hits['not_end_hit'] * truth_hits['next_hit']
    del truth_hits['next_particle']

    
    geom_cols = ['x', 'y', 'z', 'volume_id', 'layer_id', 'module_id', 'dist', 'rad_dist', 'block']
    X = truth_hits[geom_cols + cell_cols + ['next_hit']]
    #X.columns = X.columns + '_1'
    tmp = truth_hits[['hit_id'] + geom_cols + cell_cols]
    #tmp.columns = tmp.columns + '_2'
    X = pd.merge(X.rename({'next_hit':'hit_id'},axis=1), tmp, on=['hit_id'], how='left')
    del X['hit_id']


    
    tmp = truth_hits[['hit_id'] + geom_cols + cell_cols]
    tmp1 = truth_hits[['hit_id'] + geom_cols + cell_cols]
    tmp = pd.merge(tmp, tmp1, on=['volume_id', 'layer_id', 'module_id'], how='outer')
    tmp = tmp[tmp['hit_id_x'] != tmp['hit_id_y']]
    del tmp['hit_id_x']
    del tmp['hit_id_y']
    tmp['volume_id_x'] = tmp['volume_id']
    tmp['layer_id_x'] = tmp['layer_id']
    tmp['module_id_x'] = tmp['module_id']

    tmp['volume_id_y'] = tmp['volume_id']
    tmp['layer_id_y'] = tmp['layer_id']
    tmp['module_id_y'] = tmp['module_id']

    del tmp['volume_id']
    del tmp['layer_id']
    del tmp['module_id']

    tmp = tmp[X.columns]

#     tmp_idx = random.sample(list(tmp.index.values), 82782)
#     tmp = tmp.ix[tmp_idx]
    

    X['target'] = 1
    tmp['target'] = 0
    X = pd.concat([X, tmp], axis=0)

    del X['block_y']
    del X['block_x']

    X.index=range(len(X))

    X['f1'] = (X['x_x'] - X['x_y'])**2 + (X['y_x'] - X['y_y'])**2 + (X['z_x'] - X['z_y'])**2    
    X['f2'] = (X['z_x'] - X['z_y']) / (X['x_x'] - X['x_y'])
    X['f3'] = (X['z_x'] - X['z_y']) / (X['y_x'] - X['y_y'])
    
    if cnt == 0 or cnt == 1:
        X_val = pd.concat([X_val, X], axis=0)
    else:
        X_train = pd.concat([X_train, X], axis=0)
    
    print(cnt)

        
    cnt += 1

0
1
2
3
4
5
6
7
8
9


In [262]:
X_train.shape

(16495237, 32)

In [225]:
X['f1'] = (X['x_x'] - X['x_y'])**2 + (X['y_x'] - X['y_y'])**2 + (X['z_x'] - X['z_y'])**2    
X['f2'] = (X['z_x'] - X['z_y']) / (X['x_x'] - X['x_y'])
X['f3'] = (X['z_x'] - X['z_y']) / (X['y_x'] - X['y_y'])

In [151]:
blocks = truth_hits.groupby('block')['particle_id'].count().reset_index()[['block']].values.reshape(48)
block_stat = pd.DataFrame(np.zeros((48, 48)))
block_stat.index = blocks
block_stat.columns = blocks

tmp = truth_hits.groupby('particle_id')['block'].apply(lambda x: x.tolist()).reset_index()
tmp['block'].apply(lambda x: proc_arr(x))
1

In [263]:
#%%time
import lightgbm as lgbm
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score

scores = []

cat = ['module_id', 'volume_id', 'layer_id']

lgb = lgbm.sklearn.LGBMClassifier()
lgb.fit(X_train.drop(['target'] ,axis=1), X_train['target'])

pred = lgb.predict(X_val.drop(['target'],axis=1))#[:,1]


print(accuracy_score( X_val['target'], pred))

/usr/local/lib/python3.5/dist-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.9993020332165913


In [ ]:
0.9993020332165913
0.9976693708564695

In [ ]:
0.9974847311877941 6.324812597346277e-05
0.994532583391577 0.00014189783980856174

In [264]:
imp = pd.DataFrame()
imp['feature'] = X.drop(['target'], axis=1).columns.values
imp['importance'] = lgb.feature_importances_
imp = imp.sort_values('importance', ascending=False)
imp.index = range(len(imp))

In [265]:
imp

,feature,importance
0,f1,660
1,rad_dist_y,523
2,rad_dist_x,433
3,f2,209
4,dist_y,179
5,dist_x,177
6,f3,152
7,module_id_y,132
8,module_id_x,80
9,layer_id_y,64


In [204]:
1-X['target'].mean()

0.5

In [149]:
X

,x_x,y_x,z_x,volume_id_x,layer_id_x,module_id_x,dist_x,rad_dist_x,x_y,y_y,z_y,volume_id_y,layer_id_y,module_id_y,dist_y,rad_dist_y,target,fold,f1
0,-38.421902,60.531502,100.994003,8,4,283,1.534009e+04,5.140305e+03,-61.762501,97.524200,165.076996,8.0,6.0,512.0,4.057599e+04,1.332558e+04,1,1.0,6019.873047
1,-61.762501,97.524200,165.076996,8,6,512,4.057599e+04,1.332558e+04,-91.977898,145.654999,248.330994,8.0,8.0,846.0,9.134359e+04,2.967531e+04,1,2.0,10160.771484
2,-91.977898,145.654999,248.330994,8,8,846,9.134359e+04,2.967531e+04,-139.442993,221.778000,379.600006,13.0,2.0,594.0,2.127260e+05,6.862983e+04,1,0.0,25279.199219
3,-139.442993,221.778000,379.600006,13,2,594,2.127260e+05,6.862983e+04,-188.819000,301.776001,517.400024,13.0,4.0,888.0,3.944242e+05,1.267214e+05,1,4.0,27826.515625
4,-188.819000,301.776001,517.400024,13,4,888,3.944242e+05,1.267214e+05,-191.951004,306.898010,526.400024,13.0,4.0,887.0,4.081286e+05,1.310316e+05,1,0.0,117.044426
5,-191.951004,306.898010,526.400024,13,4,887,4.081286e+05,1.310316e+05,-262.704987,422.963989,725.799988,13.0,6.0,1392.0,7.746981e+05,2.479124e+05,1,2.0,58237.781250
6,-262.704987,422.963989,725.799988,13,6,1392,7.746981e+05,2.479124e+05,-345.194000,560.494019,960.599976,13.0,8.0,2024.0,1.356065e+06,4.333124e+05,1,1.0,80849.984375
7,-345.194000,560.494019,960.599976,13,8,2024,1.356065e+06,4.333124e+05,-431.626007,707.424011,1217.500000,18.0,2.0,83.0,2.169056e+06,6.867497e+05,1,2.0,95056.539062
8,-431.626007,707.424011,1217.500000,18,2,83,2.169056e+06,6.867497e+05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,2.0,NaN
9,-23.600700,21.074499,28.903299,8,2,126,1.836528e+03,1.001128e+03,-24.391100,21.806400,30.028799,8.0,2.0,127.0,1.972174e+03,1.070445e+03,1,0.0,2.427160
